In [ ]:
!pip install category_encoders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

import numpy as np
#import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score
from category_encoders import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import plot_confusion_matrix
#from sklearn.metrics import roc_curve
#from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/cardio/cardio_train.csv', sep=';')

In [ ]:
df.drop("id",axis=1, inplace=True)

In [ ]:
# 데이터셋 나누기
X = df.drop('cardio', axis=1)
y = df['cardio']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2)

In [ ]:
numeric_feats = ['age','height','weight','ap_hi','ap_lo']

scaler = StandardScaler()
X_train[numeric_feats] = scaler.fit_transform(X_train[numeric_feats])
X_val[numeric_feats] = scaler.transform(X_val[numeric_feats])
X_test[numeric_feats] = scaler.transform(X_test[numeric_feats])

In [ ]:
#원핫인코딩
ohe = OneHotEncoder(use_cat_names=True, cols = ['gender'])

X_train_ohe = ohe.fit_transform(X_train)
X_val_ohe = ohe.transform(X_val)
X_test_ohe = ohe.transform(X_test)

In [ ]:
logistic = LogisticRegression(max_iter=1000)
logistic.fit(X_train_ohe, y_train)

LogisticRegression(max_iter=1000)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, logistic.predict(X_train_ohe))

0.7219196428571428

In [ ]:
# TP TN FP FN
prediction = logistic.predict(X_test_ohe)
true_label = y_test
tmp = pd.DataFrame({'true label':true_label , 'prediction':prediction})
con1 = tmp['true label'] == tmp['prediction']
con2 = tmp['prediction'] == True

TP = len(tmp[con1 & con2])
TN = len(tmp[con1 & ~con2])
FP = len(tmp[~con1 & con2])
FN = len(tmp[~con1 & ~con2])
print("TP={},TN={},FP={},FN={}".format(TP,TN,FP,FN))

accuracy = (TP + TN) / len(y_test)
precision = TP / (TP + FP)
TPR = TP / (TP + FN) #recall
FPR = FP / (FP + TN)
print("TPR={},FPR={}".format(TPR,FPR))
recall = TPR 
f1 = 2*precision*recall/(precision+recall)

print(f'accuracy of logistic regression is {np.round(accuracy,2)}')
print(f'precision of logistic regression is {np.round(precision,2)}')
print(f'recall of logistic regression is {np.round(recall,2)}')
print(f'f1 of logistic regression is {np.round(f1,2)}')

TP=4714,TN=5424,FP=1605,FN=2257
TPR=0.676230096112466,FPR=0.2283397353819889
accuracy of logistic regression is 0.72
precision of logistic regression is 0.75
recall of logistic regression is 0.68
f1 of logistic regression is 0.71


In [ ]:
pd.DataFrame(index = ['True 0', 'True1'], data = {'Predicted 0':[TN, FN], 'Predicted 1':[FP, TP]})

,Predicted 0,Predicted 1
True 0,5424,1605
True1,2257,4714
